In [134]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal


In [135]:
llm1= ChatGroq(
    groq_api_key="gsk_13NEXVpBau9nFY8a0FNoNjfRZzctkOyZGRdji0",
     model="llama3-8b-8192"
)


In [136]:
parser = StrOutputParser()

In [143]:


class feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(
        description='Give the sentiment of the feedback, either "positive" or "negative"',
        examples=['positive']
    )


In [144]:
parser2 = PydanticOutputParser(pydantic_object=feedback)

In [145]:
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

In [146]:
classifier_chain = prompt1 | llm1 | parser2

In [147]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

In [148]:
prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)


In [149]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | llm1 | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | llm1 | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [150]:
chain = classifier_chain | branch_chain

In [151]:
print(chain.invoke({'feedback': 'This is a beautiful phone'}))



Here's a response to positive feedback:

"Thank you so much for taking the time to share your kind words! I'm thrilled to hear that you're satisfied with [specific aspect of service/product]. It's always our goal to provide the best possible experience for our customers, and I'm glad to see that we were able to meet your expectations. Your feedback is invaluable to us, and I'll make sure to pass it along to the team. Thank you again for your enthusiasm and loyalty - we appreciate it!"


In [153]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
